In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/Projeto/Silver'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
spark.sql(
    """
    create database Projeto location 'hdfs://hdfs-nn:9000/Projeto/Silver/Projeto.db'
    """
)

from pyspark.sql.types import StructType


In [ ]:
spark.sql(
    """
    DROP TABLE IF EXISTS Projeto.Harbor_Water_Quality
    """
)
​
spark.sql(
    """
    DROP DATABASE IF EXISTS Projeto CASCADE
    """
)
​

In [3]:
hdfs_path = "hdfs://hdfs-nn:9000/Projeto/bronze/Harbor_Water_Quality.csv"

Harbor_Water_Quality = spark \
            .read \
            .option("delimiter", ",") \
            .option("header", "true") \
            .option("inferSchema", True) \
            .csv(hdfs_path)

Harbor_Water_Quality.show()
Harbor_Water_Quality.printSchema()
Harbor_Water_Quality.toPandas()


+-----------------+----------------+-----------+-----------+------------------------------+---------------------------+------------------------------+----------------------+--------------------+------------------------+-------------------+----------------------+----------------------+-------------------------+---------------------------------------------------------------------------+------------------------------------------------------------------------------+-----------------------------------+--------------------------------------+---------------------------------------+------------------------------------------+-------------------+----------------------+-----------------------+------------------------+-------------------------------------------------------------+----------------------------------------------------------------+--------------------------------------------------------------------------------+------------------------+--------------------------------+------------------

,Sampling Location,Duplicate Sample,Sample Date,Sample Time,Weather Condition (Dry or Wet),Top Sample Temperature (ºC),Bottom Sample Temperature (ºC),Site Actual Depth (ft),Top Sample Depth(ft),Bottom Sample Depth (ft),...,Oakwood BOD Top Sample (mg/L),Oakwood BOD Bottom Sample(mg/L),Oakwood Total Suspended Solid Top Sample (mg/L),Oakwood Total Suspended Solid Bottom Sample (mg/L),Top Turbidity( Nephelometric Turbidity Units),Bottom Turbidity YSI (Nephelometric Turbidity Units),Sampling Comment,Long,Lat,Type
0,BR5,None,08/17/2015,1:07 PM,Dry,24.68,None,13.0,3.0,NaN,...,NaN,None,None,None,None,None,None,-73.871333,40.813667,Permanent
1,K2,None,07/06/1965,9:50 AM,None,22.8,22.2,21.0,NaN,NaN,...,NaN,None,None,None,None,None,None,-74.153000,40.641167,Permanent
2,N8,None,07/16/1909,4:10 PM,None,20,None,NaN,1.0,NaN,...,NaN,None,None,None,None,None,None,-74.045500,40.606167,Permanent
3,N9,None,07/27/1972,9:22 AM,None,17.2,16.1,NaN,NaN,NaN,...,0.6,0.6,None,None,None,None,None,-73.983333,40.568333,Permanent
4,NC0,None,02/26/2014,10:45 AM,Dry,2.51,2.46,15.0,3.0,12.0,...,NaN,None,None,None,None,None,Light snow,-73.931556,40.714961,Permanent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93702,N16,None,07/25/1974,10:13 AM,None,19.4,19.4,NaN,NaN,NaN,...,4.5,3,None,None,None,None,None,-73.943667,40.529667,Permanent
93703,TNQ,None,08/07/1978,10:05 AM,None,22.8,22.2,NaN,NaN,NaN,...,NaN,None,None,None,None,None,None,-73.79204,40.79494,Permanent
93704,J1,None,06/30/1977,11:05 AM,None,21.1,20,NaN,NaN,NaN,...,2.7,3,None,None,None,None,None,-73.884667,40.573333,Permanent
93705,J2,None,08/04/1977,12:42 PM,None,24.4,23.3,NaN,NaN,NaN,...,0.8,0.4,None,None,None,None,None,-73.885835,40.608054,Permanent


In [16]:
replaced_hwq = Harbor_Water_Quality.drop("Duplicate Sample","Top Sample Depth(ft)","Bottom Sample Depth(ft)","Top Conductivity (S/m)","Bottom Conductivity (S/m)","CTD (conductivity, temperature, depth profiler) Top Dissolved Oxygen (mg/L)","CTD (conductivity, temperature, depth profiler) Bottom Dissolved Oxygen (mg/L)","Percentage O2 Saturation Top Sample","Percentage O2 Saturation Bottom Sample","Light Trans (% transparency) Top Sample","Light Trans (% transparency) Bottom Sample","Top Sigma-T (kg/m3)","Bottom Sigma-T (kg/m3)","Top Fluorometer (mg/m3)","Bottom Fluorometer mg/m3","CTD (conductivity, temperature, depth profiler) PH Top Sample","CTD (conductivity, temperature, depth profiler) PH Bottom Sample","Top Sample Photosysthetically Active Radiation Reference (400-700nm light (uE/S)","Top Sample Par (uE/S m2)","Bottom Sample Par uE/S (uE/S m2)","Oxidation Reduction Potential (mV) Top Sample","Oxidation Reduction Potential (mV) Bottom Sample","Top Total Coliform Cells/100 mL","Top Bottom Coliform Cells/100 mL","Top Fecal Coliform Bacteria (Cells/100mL)","Bottom Fecal Coliform Bacteria (Cells/100mL)","Top Enterococci Bacteria (Cells/100mL)","Bottom Enterococci Bacteria (Cells/100mL)","Top Nitrate/Nitrite (mg/L)","Bottom Nitrate/Nitrite (mg/L)","Top Ammonium (mg/L)","Bottom Ammonium (mg/L)","Top Ortho-Phosphorus (mg/L)","Bottom Ortho-Phosphorus (mg/L)","Top Total Kjeldhal Nitrogen (mg/L)","Bottom Total Kjeldhal Nitrogen (mg/L)","Top Silica (mg/L)","Bottom Silica (mg/L)","Total Phosphorus(mg/L)","Bottom Total Phosphorus (mg/L)","Top Total Suspended Solid (mg/L)","Bottom Total Suspended Solid (mg/L)","Top Active Chlorophyll 'A' (Âµg/L)","Bottom Active Chlorophyll 'A' (Âµg/L)","Top Dissolved Organic Carbon (mg/L)","Oakwood Total Suspended Solid Bottom Sample (mg/L)","Top Turbidity( Nephelometric Turbidity Units)","Bottom Turbidity YSI (Nephelometric Turbidity Units)","Weather Condition (Dry or Wet)","current speed","Wind Speed (mph)","Sea State","Fecal Coliform Top Sample Less Than or Greater Than Result","Fecal Coliform Bottom Sample Less Than or Greater Than Result","Enterococcus Top Sample Less Than or Greater Than Result","Enterococcus Bottom Sample Less Than or Greater Than Result","Bottom Active Chlorophyll 'A' (Âµg/L)","Chlorophyll Top Sample Field (u/L (YSI)","Chlorophyll Bottom Sample Field (u/L (YSI)","Bottom Dissolved Organic Carbon (mg/L)","Top Dissolved Organic Carbon YSI (mg/L)","Bottom Dissolved Organic Carbon YSI (mg/L)","Top Sample Depth  YSI (ft)","Bottom Depth  YSI (ft)","Top Sample Salinity YSI (psu)","Bottom Sample Salinity YSI (psu)","Top Sample Temperature  YSI (ÂºC)","Bottom Sample Temperature YSI (ÂºC)","Top Bacteria # per C.C. 1909","Bottom Bacteria # per C.C. 1909","Top Beam Attenuation Coefficient","Bottom Beam Attenuation Coefficient","Top Five-Day Biochemical Oxygen Demand(mg/L)","Bottom Five-Day Biochemical Oxygen Demand(mg/L)","Top PCB Plate Count (Thous/mL)","Bottom PCB plate Count (Thous/mL)","Top Coliform (Log No/mL)","Bottom Coliform (Log No/mL)","Oakwood BOD Top Sample (mg/L)","Oakwood BOD Bottom Sample(mg/L)","Oakwood Total Suspended Solid Bottom Sample (mg/L)","Wind Direction (Wind Direction)","Bottom Sample Depth (ft)","Current Speed (knot)","Sea State","Secchi Depth (ft)","Top PH","Bottom PH","Top Active Chlorophyll 'A' (µg/L)","Bottom Active Chlorophyll 'A' (µg/L)","TOP Total Organic Carbon (mg/L)","Top PCB Plate Count (Thous/mL)","Bottom PCB plate Count (Thous/mL)","Oakwood BOD Top Sample (mg/L)","Oakwood  Total Suspended Solid Top Sample  (mg/L)","Sampling Comment","Top PCB Plate Count (Thous/mL) ","Bottom PCB plate Count (Thous/mL) ","akwood BOD Top Sample (mg/L) ","Sea State ","Top Sample Temperature  YSI (ºC)","Bottom Sample Temperature YSI (ºC)","Oakwood BOD Top Sample (mg/L) ")
replaced_hwq.printSchema()
                    
                                    

root
 |-- Sampling Location: string (nullable = true)
 |-- Sample Date: string (nullable = true)
 |-- Sample Time: string (nullable = true)
 |-- Top Sample Temperature (ºC): string (nullable = true)
 |-- Bottom Sample Temperature (ºC): string (nullable = true)
 |-- Site Actual Depth (ft): double (nullable = true)
 |-- Top Salinity  (psu): string (nullable = true)
 |-- Bottom Salinity  (psu): string (nullable = true)
 |-- Current Direction (Current Direction): string (nullable = true)
 |-- Winkler Method Top Dissolved Oxygen (mg/L): string (nullable = true)
 |-- Winkler Method Bottom Dissolved Oxygen (mg/L): string (nullable = true)
 |-- Long: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Type: string (nullable = true)



In [17]:
replaced_hwq1 = replaced_hwq.withColumn(
    "Sampling Location",
    when(
        (col("Sampling Location").isNull()), 
        "Indefinido"
    ).otherwise(col("Sampling Location")))

replaced_hwq2 = replaced_hwq1.withColumn(
    "Sample Date",
    when(
        (col("Sample Date").isNull()), 
        "Indefinido"
    ).otherwise(col("Sample Date")))

replaced_hwq3 = replaced_hwq2.withColumn(
    "Sample Time",
    when(
        (col("Sample Time").isNull()), 
        "Indefinido"
    ).otherwise(col("Sample Time")))

replaced_hwq4 = replaced_hwq3.withColumn(
    "Top Sample Temperature (ºC)",
    when(
        (col("Top Sample Temperature (ºC)").isNull()), 
        "Indefinido"
    ).otherwise(col("Top Sample Temperature (ºC)")))

replaced_hwq5 = replaced_hwq4.withColumn(
    "Bottom Sample Temperature (ºC)",
    when(
        (col("Bottom Sample Temperature (ºC)").isNull()), 
        "Indefinido"
    ).otherwise(col("Bottom Sample Temperature (ºC)")))

replaced_hwq6 = replaced_hwq5.withColumn(
    "Site Actual Depth (ft)",
    when(
        (col("Site Actual Depth (ft)").isNull()), 
        "Indefinido"
    ).otherwise(col("Site Actual Depth (ft)")))

replaced_hwq7 = replaced_hwq6.withColumn(
    "Top Salinity  (psu)",
    when(
        (col("Top Salinity  (psu)").isNull()), 
        "Indefinido"
    ).otherwise(col("Top Salinity  (psu)")))

replaced_hwq8 = replaced_hwq7.withColumn(
    "Bottom Salinity  (psu)",
    when(
        (col("Bottom Salinity  (psu)").isNull()), 
        "Indefinido"
    ).otherwise(col("Bottom Salinity  (psu)")))

replaced_hwq9 = replaced_hwq8.withColumn(
    "Current Direction (Current Direction)",
    when(
        (col("Current Direction (Current Direction)").isNull()), 
        "Indefinido"
    ).otherwise(col("Current Direction (Current Direction)")))

replaced_hwq10 = replaced_hwq9.withColumn(
    "Winkler Method Top Dissolved Oxygen (mg/L)",
    when(
        (col("Winkler Method Top Dissolved Oxygen (mg/L)").isNull()), 
        "Indefinido"
    ).otherwise(col("Winkler Method Top Dissolved Oxygen (mg/L)")))

replaced_hwq11 = replaced_hwq10.withColumn(
    "Winkler Method Bottom Dissolved Oxygen (mg/L)",
    when(
        (col("Winkler Method Bottom Dissolved Oxygen (mg/L)").isNull()), 
        "Indefinido"
    ).otherwise(col("Winkler Method Bottom Dissolved Oxygen (mg/L)")))

replaced_hwq12 = replaced_hwq11.withColumn(
    "Long",
    when(
        (col("Long").isNull()), 
        "Indefinido"
    ).otherwise(col("Long")))

replaced_hwq13 = replaced_hwq12.withColumn(
    "Lat",
    when(
        (col("Lat").isNull()), 
        "Indefinido"
    ).otherwise(col("Lat")))

replaced_hwq14 = replaced_hwq13.withColumn(
    "Type",
    when(
        (col("Type").isNull()), 
        "Indefinido"
    ).otherwise(col("Type")))



In [19]:
replaced_hwq15 = replaced_hwq14.withColumn('Year', split(replaced_hwq14['Sample Date'],'/').getItem(2))
replaced_hwq15.printSchema()
replaced_hwq15.show()

root
 |-- Sampling Location: string (nullable = true)
 |-- Sample Date: string (nullable = true)
 |-- Sample Time: string (nullable = true)
 |-- Top Sample Temperature (ºC): string (nullable = true)
 |-- Bottom Sample Temperature (ºC): string (nullable = true)
 |-- Site Actual Depth (ft): string (nullable = true)
 |-- Top Salinity  (psu): string (nullable = true)
 |-- Bottom Salinity  (psu): string (nullable = true)
 |-- Current Direction (Current Direction): string (nullable = true)
 |-- Winkler Method Top Dissolved Oxygen (mg/L): string (nullable = true)
 |-- Winkler Method Bottom Dissolved Oxygen (mg/L): string (nullable = true)
 |-- Long: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Year: string (nullable = true)

+-----------------+-----------+-----------+---------------------------+------------------------------+----------------------+-------------------+----------------------+-------------------------------------+----------

In [22]:
NewColumns = (column.replace(' ','_') for column in replaced_hwq15.columns)
replaced_hwq16 = replaced_hwq15.toDF(*NewColumns)

replaced_hwq17 = replaced_hwq16.withColumnRenamed("Top_Sample_Temperature_(ºC)","Top_Sample_Temperature_Celsius")
replaced_hwq18 = replaced_hwq17.withColumnRenamed("Bottom_Sample_Temperature_(ºC)","Bottom_Sample_Temperature_Celsius")
replaced_hwq19 = replaced_hwq18.withColumnRenamed("Top_Salinity__(psu)","Top_Salinity_psu")
replaced_hwq20 = replaced_hwq19.withColumnRenamed("Bottom_Salinity__(psu)","Bottom_Salinity_psu")
replaced_hwq21 = replaced_hwq20.withColumnRenamed("Current_Direction_(Current_Direction)","Current_Direction")
replaced_hwq22 = replaced_hwq21.withColumnRenamed("Winkler_Method_Top_Dissolved_Oxygen_(mg/L)","Winkler_Method_Top_Dissolved_Oxygen_mgL")
replaced_hwq23 = replaced_hwq22.withColumnRenamed("Winkler_Method_Bottom_Dissolved_Oxygen_(mg/L)","Winkler_Method_Bottom_Dissolved_Oxygen_mgL")
replaced_hwq24 = replaced_hwq23.withColumnRenamed("Site_Actual_Depth_(ft)","Site_Actual_Depth_ft")

In [23]:
replaced_hwq24.printSchema()

root
 |-- Sampling_Location: string (nullable = true)
 |-- Sample_Date: string (nullable = true)
 |-- Sample_Time: string (nullable = true)
 |-- Top_Sample_Temperature_Celsius: string (nullable = true)
 |-- Bottom_Sample_Temperature_Celsius: string (nullable = true)
 |-- Site_Actual_Depth_ft: string (nullable = true)
 |-- Top_Salinity_psu: string (nullable = true)
 |-- Bottom_Salinity_psu: string (nullable = true)
 |-- Current_Direction: string (nullable = true)
 |-- Winkler_Method_Top_Dissolved_Oxygen_mgL: string (nullable = true)
 |-- Winkler_Method_Bottom_Dissolved_Oxygen_mgL: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Year: string (nullable = true)



In [26]:
spark.sql(
    """
    CREATE EXTERNAL TABLE Projeto.Harbor_Water_Quality( 
     Sampling_Location VARCHAR(100),
     Sample_Date VARCHAR(50),
     Sample_Time VARCHAR(50),
     Top_Sample_Temperature_Celsius VARCHAR(50),
     Bottom_Sample_Temperature_Celsius VARCHAR(50),
     Site_Actual_Depth_ft VARCHAR(50),
     Top_Salinity_psu  VARCHAR(100),
     Bottom_Salinity_psu  VARCHAR(100),
     Current_Direction  VARCHAR(100),
     Winkler_Method_Top_Dissolved_Oxygen_mgL  VARCHAR(100),
     Winkler_Method_Bottom_Dissolved_Oxygen_mgL  VARCHAR(100), 
     Long VARCHAR(50),
     Lat  VARCHAR(100),
     Type VARCHAR(50)
        )
    USING DELTA
    PARTITIONED BY (
        Year  VARCHAR(100)

    )
    LOCATION 'hdfs://hdfs-nn:9000/Projeto/Silver/Projeto.db/Harbor_Water_Quality'
    """
    )

DataFrame[]

In [27]:
replaced_hwq24.show()
replaced_hwq24 \
    .select("Sampling_Location","Sample_Date","Sample_Time","Top_Sample_Temperature_Celsius","Bottom_Sample_Temperature_Celsius","Site_Actual_Depth_ft",
            "Top_Salinity_psu","Bottom_Salinity_psu","Current_Direction","Winkler_Method_Top_Dissolved_Oxygen_mgL","Winkler_Method_Bottom_Dissolved_Oxygen_mgL","Long",
            "Lat", "Type","Year") \
    .write \
    .mode("overwrite") \
    .partitionBy("Year") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/Projeto/Silver/Projeto.db/Harbor_Water_Quality")

+-----------------+-----------+-----------+------------------------------+---------------------------------+--------------------+----------------+-------------------+-----------------+---------------------------------------+------------------------------------------+----------+---------+---------+----+
|Sampling_Location|Sample_Date|Sample_Time|Top_Sample_Temperature_Celsius|Bottom_Sample_Temperature_Celsius|Site_Actual_Depth_ft|Top_Salinity_psu|Bottom_Salinity_psu|Current_Direction|Winkler_Method_Top_Dissolved_Oxygen_mgL|Winkler_Method_Bottom_Dissolved_Oxygen_mgL|      Long|      Lat|     Type|Year|
+-----------------+-----------+-----------+------------------------------+---------------------------------+--------------------+----------------+-------------------+-----------------+---------------------------------------+------------------------------------------+----------+---------+---------+----+
|              BR5| 08/17/2015|    1:07 PM|                         24.68|              

In [28]:
spark.table("Projeto.Harbor_Water_Quality").toPandas()

,Sampling_Location,Sample_Date,Sample_Time,Top_Sample_Temperature_Celsius,Bottom_Sample_Temperature_Celsius,Site_Actual_Depth_ft,Top_Salinity_psu,Bottom_Salinity_psu,Current_Direction,Winkler_Method_Top_Dissolved_Oxygen_mgL,Winkler_Method_Bottom_Dissolved_Oxygen_mgL,Long,Lat,Type,Year
0,E8,09/11/2017,2:19 PM,21.05,0.00,81.0,25.41,26.15,E,5.85,5.76,-73.8205,40.805,Permanent,2017
1,WC3,09/05/2017,12:42 PM,Indefinido,Indefinido,14.0,Indefinido,Indefinido,N,Indefinido,Indefinido,-73.84319,40.81507,Permanent,2017
2,AC1,08/10/2017,11:15 AM,Indefinido,Indefinido,3.93,Indefinido,Indefinido,Indefinido,Indefinido,Indefinido,-73.75175,40.76317,Permanent,2017
3,E15,09/05/2017,3:19 PM,22.09,21.86,13.0,24.00,24.05,N,5.60,Indefinido,-73.8495,40.763,Permanent,2017
4,K4,06/06/2017,12:14 PM,18.10,17.83,36.0,18.61,19.58,S,6.14,6.04,-74.21167,40.567,Permanent,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93702,BR3,10/22/2020,9:14 AM,Indefinido,Indefinido,4.45,Indefinido,Indefinido,Indefinido,Indefinido,Indefinido,-73.8848,"40.82537,,,,,,,,,,,,,,,,,,,,,,,,,",Permanent,2020
93703,E11,10/06/2020,10:56 AM,19.33,Indefinido,6.0,26.10,Indefinido,S,7.61,Indefinido,-73.76533,"40.84367,,,,,,,,,,,,,,,,,,,,,,,,,",Permanent,2020
93704,N3C,06/21/2021,11:09 AM,Indefinido,Indefinido,38.0,Indefinido,Indefinido,S,Indefinido,Indefinido,-73.9815,"40.79317,",Provisional,2021
93705,K2,01/21/2021,11:59 AM,Indefinido,Indefinido,29.0,Indefinido,Indefinido,W,Indefinido,Indefinido,-74.153,"40.64117,",Provisional,2021
